In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import chroma, FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


chache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 500,
    chunk_overlap = 100,
)
loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter = splitter)

embeddings = OpenAIEmbeddings()

#chaching embeddings
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, chache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


In [21]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm = llm,
    max_token_limit=150,
    return_messages=True,

)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
         You are a helpful assistant. Answer question only using the following context. If you don't know the answer just say you don't know.
         :\n\n 
         {context}
         """),
         MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = {"context":retriever, "question": RunnablePassthrough(), "history":load_memory}  | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input":question},
        {"output": result.content},
    )
    print(result)

In [22]:
question1 = "Aaronson 은 유죄인가요?"
invoke_chain(question1)


content='모르겠습니다.'


In [23]:
question2 =  "그가 테이블에 어떤 메시지를 썼나요?"
invoke_chain(question2)


content='Aaronson이 테이블에 쓴 메시지는 "FREEDOM IS SLAVERY", "TWO AND TWO MAKE FIVE", 그리고 "GOD IS POWER"입니다.'


In [24]:
question3 =  "Julia 는 누구인가요?"
invoke_chain(question3)

content='Julia는 Winston의 사랑하는 사람이며, 그와 함께 자유롭고 반항적인 관계를 맺고 있던 인물입니다.'
